## Select the best model from the grid and create the stacked ensembles


In [28]:
h2o.shutdown()

In [6]:
import pandas as pd
import random

import warnings
warnings.filterwarnings('ignore')


import h2o
h2o.init(min_mem_size='27G')

DATA_LOCATION = "../../data/"
MODELS_LOCATION = "../../models/ALL_FEATURES/"

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_265"; OpenJDK Runtime Environment (build 1.8.0_265-8u265-b01-0ubuntu2~16.04-b01); OpenJDK 64-Bit Server VM (build 25.265-b01, mixed mode)
  Starting server from /anaconda/envs/azureml_py36/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpq82jmkm2
  JVM stdout: /tmp/tmpq82jmkm2/h2o_azureuser_started_from_python.out
  JVM stderr: /tmp/tmpq82jmkm2/h2o_azureuser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,5 months and 12 days !!!
H2O_cluster_name:,H2O_from_python_azureuser_bxhj89
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,25.88 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [7]:
train = h2o.import_file( DATA_LOCATION + "processed/final.train.tsv")
train.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%


SampleID,NC000962_3.22,NC000962_3.434,NC000962_3.524,NC000962_3.645,NC000962_3.648,NC000962_3.654,NC000962_3.666,NC000962_3.675,NC000962_3.678,NC000962_3.693,NC000962_3.698,NC000962_3.699,NC000962_3.702,NC000962_3.705,NC000962_3.708,NC000962_3.717,NC000962_3.729,NC000962_3.741,NC000962_3.744,NC000962_3.747,NC000962_3.750,NC000962_3.756,NC000962_3.770,NC000962_3.777,NC000962_3.780,NC000962_3.783,NC000962_3.793,NC000962_3.795,NC000962_3.799,NC000962_3.801,NC000962_3.805,NC000962_3.822,NC000962_3.840,NC000962_3.846,NC000962_3.849,NC000962_3.852,NC000962_3.1045,NC000962_3.1049,NC000962_3.1089,NC000962_3.1123,NC000962_3.1152,NC000962_3.1153,NC000962_3.1155,NC000962_3.1161,NC000962_3.1164,NC000962_3.1166,NC000962_3.1167,NC000962_3.1176,NC000962_3.1206,NC000962_3.1212,NC000962_3.1255,NC000962_3.1278,NC000962_3.1291,NC000962_3.1302,NC000962_3.1326,NC000962_3.1389,NC000962_3.1399,NC000962_3.1413,NC000962_3.1416,NC000962_3.1422,NC000962_3.1429,NC000962_3.1431,NC000962_3.1432,NC000962_3.1452,NC000962_3.1458,NC000962_3.1461,NC000962_3.1470,NC000962_3.1473,NC000962_3.1474,NC000962_3.1653,NC000962_3.1676,NC000962_3.1699,NC000962_3.1703,NC000962_3.1708,NC000962_3.1718,NC000962_3.1729,NC000962_3.1771,NC000962_3.1827,NC000962_3.1849,NC000962_3.1918,NC000962_3.1977,NC000962_3.2532,NC000962_3.2745,NC000962_3.3352,NC000962_3.3446,NC000962_3.4013,NC000962_3.4086,NC000962_3.4096,NC000962_3.4119,NC000962_3.4938,NC000962_3.5075,NC000962_3.5627,NC000962_3.5782,NC000962_3.5790,NC000962_3.5791,NC000962_3.5803,NC000962_3.5807,NC000962_3.5824,NC000962_3.5839,NC000962_3.5848,NC000962_3.5856,NC000962_3.5858,NC000962_3.5860,NC000962_3.5902,NC000962_3.6003,NC000962_3.6013,NC000962_3.6112,NC000962_3.6280,NC000962_3.6286,NC000962_3.6292,NC000962_3.6307,NC000962_3.6362,NC000962_3.6382,NC000962_3.6388,NC000962_3.6403,NC000962_3.6430,NC000962_3.6436,NC000962_3.6439,NC000962_3.6445,NC000962_3.6452,NC000962_3.6453,NC000962_3.6502,NC000962_3.6508,NC000962_3.6511,NC000962_3.6515,NC000962_3.6520,NC000962_3.6535,NC000962_3.6547,NC000962_3.6550,NC000962_3.6551,NC000962_3.6553,NC000962_3.6571,NC000962_3.6575,NC000962_3.6579,NC000962_3.6586,NC000962_3.6620,NC000962_3.6638,NC000962_3.6695,NC000962_3.6735,NC000962_3.6738,NC000962_3.6742,NC000962_3.6749,NC000962_3.6750,NC000962_3.6807,NC000962_3.6878,NC000962_3.6881,NC000962_3.7058,NC000962_3.7088,NC000962_3.7170,NC000962_3.7268,NC000962_3.7355,NC000962_3.7362,NC000962_3.7496,NC000962_3.7563,NC000962_3.7564,NC000962_3.7566,NC000962_3.7567,NC000962_3.7570,NC000962_3.7572,NC000962_3.7581,NC000962_3.7582,NC000962_3.7585,NC000962_3.7607,NC000962_3.7631,NC000962_3.7637,NC000962_3.7652,NC000962_3.7658,NC000962_3.7664,NC000962_3.7683,NC000962_3.7685,NC000962_3.7694,NC000962_3.7710,NC000962_3.7712,NC000962_3.7725,NC000962_3.7728,NC000962_3.7730,NC000962_3.7892,NC000962_3.8040,NC000962_3.8164,NC000962_3.8201,NC000962_3.8434,NC000962_3.8452,NC000962_3.8519,NC000962_3.8619,NC000962_3.8624,NC000962_3.9023,NC000962_3.9032,NC000962_3.9034,NC000962_3.9050,NC000962_3.9051,NC000962_3.9113,NC000962_3.9119,NC000962_3.9134,NC000962_3.9143,NC000962_3.9145,NC000962_3.9147,NC000962_3.9153,NC000962_3.9154,NC000962_3.9155
SRR10525336,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
SRR10380004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
SRR6807701,0,0,0,0,0,0,0

In [8]:

test = h2o.import_file( DATA_LOCATION + "processed/final.test.tsv")
test.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%


SampleID,NC000962_3.22,NC000962_3.434,NC000962_3.524,NC000962_3.645,NC000962_3.648,NC000962_3.654,NC000962_3.666,NC000962_3.675,NC000962_3.678,NC000962_3.693,NC000962_3.698,NC000962_3.699,NC000962_3.702,NC000962_3.705,NC000962_3.708,NC000962_3.717,NC000962_3.729,NC000962_3.741,NC000962_3.744,NC000962_3.747,NC000962_3.750,NC000962_3.756,NC000962_3.770,NC000962_3.777,NC000962_3.780,NC000962_3.783,NC000962_3.793,NC000962_3.795,NC000962_3.799,NC000962_3.801,NC000962_3.805,NC000962_3.822,NC000962_3.840,NC000962_3.846,NC000962_3.849,NC000962_3.852,NC000962_3.1045,NC000962_3.1049,NC000962_3.1089,NC000962_3.1123,NC000962_3.1152,NC000962_3.1153,NC000962_3.1155,NC000962_3.1161,NC000962_3.1164,NC000962_3.1166,NC000962_3.1167,NC000962_3.1176,NC000962_3.1206,NC000962_3.1212,NC000962_3.1255,NC000962_3.1278,NC000962_3.1291,NC000962_3.1302,NC000962_3.1326,NC000962_3.1389,NC000962_3.1399,NC000962_3.1413,NC000962_3.1416,NC000962_3.1422,NC000962_3.1429,NC000962_3.1431,NC000962_3.1432,NC000962_3.1452,NC000962_3.1458,NC000962_3.1461,NC000962_3.1470,NC000962_3.1473,NC000962_3.1474,NC000962_3.1653,NC000962_3.1676,NC000962_3.1699,NC000962_3.1703,NC000962_3.1708,NC000962_3.1718,NC000962_3.1729,NC000962_3.1771,NC000962_3.1827,NC000962_3.1849,NC000962_3.1918,NC000962_3.1977,NC000962_3.2532,NC000962_3.2745,NC000962_3.3352,NC000962_3.3446,NC000962_3.4013,NC000962_3.4086,NC000962_3.4096,NC000962_3.4119,NC000962_3.4938,NC000962_3.5075,NC000962_3.5627,NC000962_3.5782,NC000962_3.5790,NC000962_3.5791,NC000962_3.5803,NC000962_3.5807,NC000962_3.5824,NC000962_3.5839,NC000962_3.5848,NC000962_3.5856,NC000962_3.5858,NC000962_3.5860,NC000962_3.5902,NC000962_3.6003,NC000962_3.6013,NC000962_3.6112,NC000962_3.6280,NC000962_3.6286,NC000962_3.6292,NC000962_3.6307,NC000962_3.6362,NC000962_3.6382,NC000962_3.6388,NC000962_3.6403,NC000962_3.6430,NC000962_3.6436,NC000962_3.6439,NC000962_3.6445,NC000962_3.6452,NC000962_3.6453,NC000962_3.6502,NC000962_3.6508,NC000962_3.6511,NC000962_3.6515,NC000962_3.6520,NC000962_3.6535,NC000962_3.6547,NC000962_3.6550,NC000962_3.6551,NC000962_3.6553,NC000962_3.6571,NC000962_3.6575,NC000962_3.6579,NC000962_3.6586,NC000962_3.6620,NC000962_3.6638,NC000962_3.6695,NC000962_3.6735,NC000962_3.6738,NC000962_3.6742,NC000962_3.6749,NC000962_3.6750,NC000962_3.6807,NC000962_3.6878,NC000962_3.6881,NC000962_3.7058,NC000962_3.7088,NC000962_3.7170,NC000962_3.7268,NC000962_3.7355,NC000962_3.7362,NC000962_3.7496,NC000962_3.7563,NC000962_3.7564,NC000962_3.7566,NC000962_3.7567,NC000962_3.7570,NC000962_3.7572,NC000962_3.7581,NC000962_3.7582,NC000962_3.7585,NC000962_3.7607,NC000962_3.7631,NC000962_3.7637,NC000962_3.7652,NC000962_3.7658,NC000962_3.7664,NC000962_3.7683,NC000962_3.7685,NC000962_3.7694,NC000962_3.7710,NC000962_3.7712,NC000962_3.7725,NC000962_3.7728,NC000962_3.7730,NC000962_3.7892,NC000962_3.8040,NC000962_3.8164,NC000962_3.8201,NC000962_3.8434,NC000962_3.8452,NC000962_3.8519,NC000962_3.8619,NC000962_3.8624,NC000962_3.9023,NC000962_3.9032,NC000962_3.9034,NC000962_3.9050,NC000962_3.9051,NC000962_3.9113,NC000962_3.9119,NC000962_3.9134,NC000962_3.9143,NC000962_3.9145,NC000962_3.9147,NC000962_3.9153,NC000962_3.9154,NC000962_3.9155
ERR3335735,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
SRR8552929,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,1,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0
ERR067629,0,0,0,0,0,0,0,0,

In [9]:
# Identify predictors and response
train_predictor_cols = train.columns
train_response_col = "Resistance_Status"
train_predictor_cols.remove('SampleID')
train_predictor_cols.remove(train_response_col)
print("train frame - predictor column: ", train_predictor_cols[0], train_predictor_cols[-1])
print("train frame - response column: ", train_response_col)



# Identify predictors and response
test_predictor_cols = test.columns
test_response_col = "Resistance_Status"
test_predictor_cols.remove('SampleID')
test_predictor_cols.remove(test_response_col)
print("test frame - predictor columns: ", test_predictor_cols[0], test_predictor_cols[-1])
print("test frame - response column: ", test_response_col)

train frame - predictor column:  NC000962_3.22 NC000962_3.4411327
train frame - response column:  Resistance_Status
test frame - predictor columns:  NC000962_3.22 NC000962_3.4411327
test frame - response column:  Resistance_Status


In [10]:

# For binary classification, response should be a factor
train[train_response_col] = train[train_response_col].asfactor()
test[test_response_col] = test[test_response_col].asfactor()


# Number of CV folds (to generate level-one data for stacking)
nfolds = 5

MAX_GRID_MODELS = 10


x = train_predictor_cols
y = train_response_col




# Stacked ensemble from grid models

In [17]:
nb_grid = h2o.load_grid(MODELS_LOCATION + "FINAL/nb_grid/Grid_NaiveBayes_py_3_sid_9644_model_python_1604419067081_1")



glm_grid = h2o.load_grid(MODELS_LOCATION + "FINAL/glm_grid/Grid_GLM_py_3_sid_b7a1_model_python_1604419221083_1")



gbm_grid = h2o.load_grid(MODELS_LOCATION + "FINAL/gbm_grid/Grid_GBM_py_7_sid_9651_model_python_1604407520638_1")



xgb_grid = h2o.load_grid(MODELS_LOCATION + "FINAL/xgb_grid/Grid_XGBoost_py_7_sid_a3b5_model_python_1604427337744_1")



dl_grid = h2o.load_grid(MODELS_LOCATION + "FINAL/dl_grid/Grid_DeepLearning_py_3_sid_b7a1_model_python_1604419221083_608")


drf_grid = h2o.load_grid(MODELS_LOCATION + "FINAL/drf_grid/Grid_DRF_py_3_sid_9421_model_python_1604478808297_199")


In [18]:
# Select best models from the grids based on performance on the test data

def best_model_from_grid (model_grid):

    sorted_grid = model_grid.get_grid(sort_by='auc', decreasing=True)

    for mdl in sorted_grid:
        print("Modeld ID: ", mdl.model_id)

        # print('Train data AUC: ', mdl.model_performance(train=True).auc()) # same result with model_performance()
        print('Default Test data AUC: ', mdl.model_performance(valid=True).auc())    
        print('Default Test data AUCPR: ', mdl.model_performance(valid=True).aucpr())    

        print('Default Cross-validation AUC: ', mdl.model_performance(xval=True).auc())
        print('Default Cross-validation AUCPR: ', mdl.model_performance(xval=True).aucpr())
        print("\n--------------------\n")

    print("\n@@@@@@@@@@@@@@@@@@@@@@@\n")
    return sorted_grid[0]


In [26]:
drf_grid.models[0]

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_1604478808297_450_cv_1


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,10.0,10.0,17590.0,20.0,20.0,20.0,107.0,156.0,132.8




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.14854060083015827
RMSE: 0.38540965326540316
LogLoss: 1.632880988000612
Mean Per-Class Error: 0.22245192421411164
AUC: 0.8408259810298744
AUCPR: 0.8812084697658065
Gini: 0.6816519620597488

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5: 

Maximum Metrics: Maximum metrics at their respective thresholds

Gains/Lift Table: Avg response rate: 65.20 %, avg score: 65.45 %


ModelMetricsBinomial: drf
** Reported on validation data. **

MSE: 0.12097531574339374
RMSE: 0.3478150596845883
LogLoss: 0.36992736421008454
Mean Per-Class Error: 0.16309523809523818
AUC: 0.9059523809523808
AUCPR: 0.95445042368508
Gini: 0.8119047619047617

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.46638131365180013: 

Maximum Metrics: Maximum metrics at their respective thresholds


,,0,1,Error,Rate
0,0,213.0,122.0,0.3642,(122.0/335.0)
1,1,57.0,574.0,0.0903,(57.0/631.0)
2,Total,270.0,696.0,0.1853,(179.0/966.0)


,metric,threshold,value,idx
0,max f1,0.500000,0.865109,302.0
1,max f2,0.295455,0.912254,349.0
2,max f0point5,0.689135,0.848890,227.0
3,max accuracy,0.500000,0.814700,302.0
4,max precision,0.971569,0.922794,26.0
5,max recall,0.000000,1.000000,399.0
6,max specificity,1.000000,0.940299,0.0
7,max absolute_mcc,0.500000,0.578543,302.0
8,max min_per_class_accuracy,0.670030,0.773376,237.0
9,max mean_per_class_accuracy,0.687627,0.777548,230.0


...

,metric,threshold,value,idx
0,max f1,0.466381,0.872832,128.0
1,max f2,0.135407,0.928652,166.0
2,max f0point5,0.750000,0.904335,76.0
3,max accuracy,0.658554,0.831325,97.0
4,max precision,1.000000,1.000000,0.0
5,max recall,0.102074,1.000000,172.0
6,max specificity,1.000000,1.000000,0.0
7,max absolute_mcc,0.658554,0.645694,97.0
8,max min_per_class_accuracy,0.648669,0.830303,99.0
9,max mean_per_class_accuracy,0.750000,0.836905,76.0



Gains/Lift Table: Avg response rate: 66.27 %, avg score: 65.14 %


Scoring History: 

Variable Importances: 

See the whole table with table.as_data_frame()


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.084337,1.000000,1.509091,1.509091,1.000000,1.000000,1.000000,1.000000,0.127273,0.127273,50.909091,50.909091
1,,2,0.116466,0.994286,1.509091,1.509091,1.000000,0.994286,1.000000,0.998424,0.048485,0.175758,50.909091,50.909091
2,,3,0.156627,0.969286,1.509091,1.509091,1.000000,0.978104,1.000000,0.993213,0.060606,0.236364,50.909091,50.909091
3,,4,0.200803,0.942714,1.509091,1.509091,1.000000,0.952259,1.000000,0.984203,0.066667,0.303030,50.909091,50.909091
4,,5,0.305221,0.894286,1.509091,1.509091,1.000000,0.908060,1.000000,0.958154,0.157576,0.460606,50.909091,50.909091
5,,6,0.401606,0.832877,1.383333,1.478909,0.916667,0.862740,0.980000,0.935255,0.133333,0.593939,38.333333,47.890909
6,,7,0.502008,0.754134,1.328000,1.448727,0.880000,0.791209,0.960000,0.906446,0.133333,0.727273,32.800000,44.872727
7,,8,0.598394,0.658377,1.006061,1.377425,0.666667,0.704861,0.912752,0.873976,0.096970,0.824242,0.606061,37.742526
8,,9,0.698795,0.528021,0.603636,1.266249,0.400000,0.596221,0.839080,0.834068,0.060606,0.884848,-39.636364,26.624869
9,,10,0.799197,0.305455,0.603636,1.183006,0.400000,0.417113,0.783920,0.781687,0.060606,0.945455,-39.636364,18.300594


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
0,,2020-11-04 09:18:43,3.645 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2020-11-04 09:19:26,46.679 sec,2.0,0.437145,4.485478,0.787225,0.851476,1.347389,0.213703,0.420909,1.673104,0.812698,0.883247,1.434873,0.204819
2,,2020-11-04 09:20:10,1 min 31.329 sec,4.0,0.428112,3.616747,0.787729,0.844835,1.328624,0.213429,0.365724,0.650802,0.875397,0.927825,1.473996,0.180723
3,,2020-11-04 09:21:11,2 min 32.404 sec,6.0,0.403791,2.619278,0.818489,0.865389,1.367601,0.198697,0.348774,0.369123,0.896753,0.946891,1.509091,0.168675
4,,2020-11-04 09:23:08,4 min 29.263 sec,8.0,0.390734,1.951835,0.834246,0.876969,1.388927,0.188285,0.346811,0.368752,0.903427,0.950984,1.509091,0.168675
5,,2020-11-04 09:25:35,6 min 55.688 sec,10.0,0.385410,1.632881,0.840826,0.881208,1.387078,0.185300,0.347815,0.369927,0.905952,0.954450,1.509091,0.176707


,variable,relative_importance,scaled_importance,percentage
0,NC000962_3.3745733,18.911167,1.000000,0.016210
1,NC000962_3.3502231,15.631781,0.826590,0.013399
2,NC000962_3.2866569,15.494915,0.819353,0.013281
3,NC000962_3.3934734,14.527585,0.768201,0.012452
4,NC000962_3.2262026,14.254182,0.753744,0.012218
5,NC000962_3.832109,13.684945,0.723644,0.011730
6,NC000962_3.336050,13.200030,0.698002,0.011314
7,NC000962_3.1634609,13.014751,0.688205,0.011156
8,NC000962_3.336344,12.434352,0.657514,0.010658
9,NC000962_3.336611,12.127708,0.641299,0.010395


In [19]:
best_nb_model = best_model_from_grid(nb_grid)
best_glm_model = best_model_from_grid(glm_grid)
best_gbm_model = best_model_from_grid(gbm_grid)
best_xgb_model= best_model_from_grid(xgb_grid)
best_drf_model= best_model_from_grid(drf_grid)
best_dl_model= best_model_from_grid(dl_grid)

Modeld ID:  Grid_NaiveBayes_py_3_sid_9644_model_python_1604419067081_1_model_2
Default Test data AUC:  0.6037862562166479
Default Test data AUCPR:  0.7338591199790186
Default Cross-validation AUC:  0.7146276595744682
Default Cross-validation AUCPR:  0.7825978708385057

--------------------

Modeld ID:  Grid_NaiveBayes_py_3_sid_9644_model_python_1604419067081_1_model_8
Default Test data AUC:  0.5874142374850255
Default Test data AUCPR:  0.7206948614316347
Default Cross-validation AUC:  0.7076684397163121
Default Cross-validation AUCPR:  0.7874589576147568

--------------------

Modeld ID:  Grid_NaiveBayes_py_3_sid_9644_model_python_1604419067081_1_model_4
Default Test data AUC:  0.6308581696736486
Default Test data AUCPR:  0.7700108089513915
Default Cross-validation AUC:  0.6892390661938534
Default Cross-validation AUCPR:  0.7895239621957413

--------------------

Modeld ID:  Grid_NaiveBayes_py_3_sid_9644_model_python_1604419067081_1_model_7
Default Test data AUC:  0.5496424293026464
De

AttributeError: 'NoneType' object has no attribute 'auc'

In [9]:

def extract_params_from_model(actual_params_dict, extra_params = [], additional_keys = {}):
    final_params = actual_params_dict

    columns_to_be_removed =   [
                                'model_id',
                                'validation_frame',
                                'response_column',
                                'ignored_columns',
                                'training_frame',
                                *extra_params
]

    for col_name in columns_to_be_removed:
        del  final_params[col_name]

    return {**final_params, **additional_keys}

In [10]:
from h2o.estimators import H2ONaiveBayesEstimator

top_nb = H2ONaiveBayesEstimator(**extract_params_from_model(best_nb_model.actual_params))

top_nb.train(x=x, y=y, training_frame=train, validation_frame=test)

# h2o.save_model(top_nb, MODELS_LOCATION + "FINAL/top_nb")
test_perf = top_nb.model_performance(test)

print('AUC on test data: ', test_perf.auc(), "\n\n============================")

test_perf

naivebayes Model Build progress: |████████████████████████████████████████| 100%
AUC on test data:  0.6308581696736486 


ModelMetricsBinomial: naivebayes
** Reported on test data. **

MSE: 0.33561203828422775
RMSE: 0.5793203244183892
LogLoss: 4.38842128604827
Mean Per-Class Error: 0.3850328529422442
AUC: 0.6308581696736486
AUCPR: 0.7700108089513915
Gini: 0.2617163393472972

Confusion Matrix (Act/Pred) for max f1 @ threshold = 9.018296132152746e-17: 


,,0,1,Error,Rate
0,0,4.0,159.0,0.9755,(159.0/163.0)
1,1,2.0,336.0,0.0059,(2.0/338.0)
2,Total,6.0,495.0,0.3214,(161.0/501.0)



Maximum Metrics: Maximum metrics at their respective thresholds

Gains/Lift Table: Avg response rate: 67.47 %, avg score: 72.82 %



,metric,threshold,value,idx
0,max f1,9.018296e-17,0.806723,351.0
1,max f2,1.103935e-27,0.912035,356.0
2,max f0point5,5.856787e-04,0.737212,270.0
3,max accuracy,4.350758e-16,0.678643,349.0
4,max precision,9.999998e-01,0.846715,4.0
5,max recall,1.103935e-27,1.000000,356.0
6,max specificity,1.000000e+00,0.877301,0.0
7,max absolute_mcc,9.999947e-01,0.226567,30.0
8,max min_per_class_accuracy,9.972941e-01,0.588957,123.0
9,max mean_per_class_accuracy,9.999947e-01,0.614967,30.0


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.035928,1.000000e+00,0.823471,0.823471,0.555556,1.000000e+00,0.555556,1.000000,0.029586,0.029586,-17.652860,-17.652860
1,,2,0.041916,1.000000e+00,0.988166,0.846999,0.666667,1.000000e+00,0.571429,1.000000,0.005917,0.035503,-1.183432,-15.300085
2,,3,0.051896,1.000000e+00,1.482249,0.969162,1.000000,1.000000e+00,0.653846,1.000000,0.014793,0.050296,48.224852,-3.083751
3,,4,0.101796,1.000000e+00,1.304379,1.133484,0.880000,1.000000e+00,0.764706,1.000000,0.065089,0.115385,30.437870,13.348416
4,,5,0.151697,1.000000e+00,1.422959,1.228706,0.960000,1.000000e+00,0.828947,1.000000,0.071006,0.186391,42.295858,22.870601
5,,6,0.201597,1.000000e+00,1.126509,1.203410,0.760000,1.000000e+00,0.811881,1.000000,0.056213,0.242604,12.650888,20.340969
6,,7,0.301397,9.999991e-01,1.245089,1.217211,0.840000,9.999998e-01,0.821192,1.000000,0.124260,0.366864,24.508876,21.721071
7,,8,0.401198,9.999738e-01,0.978284,1.157776,0.660000,9.999905e-01,0.781095,0.999998,0.097633,0.464497,-2.171598,15.777621
8,,9,0.500998,9.995045e-01,0.948639,1.116115,0.640000,9.998360e-01,0.752988,0.999965,0.094675,0.559172,-5.136095,11.611542
9,,10,0.600798,9.756694e-01,0.978284,1.093220,0.660000,9.911600e-01,0.737542,0.998503,0.097633,0.656805,-2.171598,9.321984


In [11]:
from h2o.estimators import H2OGeneralizedLinearEstimator

top_glm = H2OGeneralizedLinearEstimator(**extract_params_from_model(best_glm_model.actual_params, ['lambda']))

top_glm.train(x=x, y=y, training_frame=train, validation_frame=test)

# h2o.save_model(top_glm, MODELS_LOCATION + "FINAL/top_glm")

test_perf = top_glm.model_performance(test)

print('AUC on test data: ', test_perf.auc(), "\n\n============================")

test_perf

glm Model Build progress: |███████████████████████████████████████████████| 100%
AUC on test data:  0.7338639416270374 


ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.1962612315157875
RMSE: 0.4430138051074566
LogLoss: 0.5630769490236587
Null degrees of freedom: 500
Residual degrees of freedom: 354
Null deviance: 633.0394016859412
Residual deviance: 564.203102921706
AIC: 858.203102921706
AUC: 0.7338639416270374
AUCPR: 0.8728658954340602
Gini: 0.46772788325407477

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.08972008272326944: 

Maximum Metrics: Maximum metrics at their respective thresholds

Gains/Lift Table: Avg response rate: 67.47 %, avg score: 65.15 %



,,0,1,Error,Rate
0,0,5.0,158.0,0.9693,(158.0/163.0)
1,1,0.0,338.0,0.0,(0.0/338.0)
2,Total,5.0,496.0,0.3154,(158.0/501.0)


,metric,threshold,value,idx
0,max f1,0.089720,0.810552,314.0
1,max f2,0.089720,0.914502,314.0
2,max f0point5,0.743340,0.800885,106.0
3,max accuracy,0.417132,0.684631,246.0
4,max precision,0.982788,1.000000,0.0
5,max recall,0.089720,1.000000,314.0
6,max specificity,0.982788,1.000000,0.0
7,max absolute_mcc,0.743340,0.413224,106.0
8,max min_per_class_accuracy,0.658888,0.671598,140.0
9,max mean_per_class_accuracy,0.743340,0.715604,106.0


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.011976,0.964324,1.482249,1.482249,1.000000,0.974854,1.000000,0.974854,0.017751,0.017751,48.224852,48.224852
1,,2,0.021956,0.954236,1.482249,1.482249,1.000000,0.956000,1.000000,0.966284,0.014793,0.032544,48.224852,48.224852
2,,3,0.031936,0.946618,1.482249,1.482249,1.000000,0.950858,1.000000,0.961464,0.014793,0.047337,48.224852,48.224852
3,,4,0.041916,0.941936,1.482249,1.482249,1.000000,0.943595,1.000000,0.957209,0.014793,0.062130,48.224852,48.224852
4,,5,0.057884,0.940769,1.482249,1.482249,1.000000,0.940769,1.000000,0.952674,0.023669,0.085799,48.224852,48.224852
5,,6,0.101796,0.899943,1.414874,1.453185,0.954545,0.920113,0.980392,0.938628,0.062130,0.147929,41.487359,45.318482
6,,7,0.165669,0.895638,1.435928,1.446532,0.968750,0.896384,0.975904,0.922341,0.091716,0.239645,43.592825,44.653169
7,,8,0.225549,0.887078,1.383432,1.429780,0.933333,0.888139,0.964602,0.913261,0.082840,0.322485,38.343195,42.977955
8,,9,0.301397,0.844087,1.326222,1.403719,0.894737,0.867856,0.947020,0.901834,0.100592,0.423077,32.622236,40.371880
9,,10,0.403194,0.740708,1.133484,1.335491,0.764706,0.801076,0.900990,0.876395,0.115385,0.538462,13.348416,33.549124


In [12]:
from h2o.estimators import H2OGradientBoostingEstimator

top_gbm = H2OGradientBoostingEstimator(**extract_params_from_model(best_gbm_model.actual_params))

top_gbm.train(x=x, y=y, training_frame=train, validation_frame=test)

# h2o.save_model(top_gbm, MODELS_LOCATION + "FINAL/top_gbm")

test_perf = top_gbm.model_performance(test)

print('AUC on test data: ', test_perf.auc(), "\n\n============================")

test_perf

gbm Model Build progress: |███████████████████████████████████████████████| 100%
AUC on test data:  0.684212436925981 


ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.20237251264178333
RMSE: 0.4498583250777775
LogLoss: 0.5892686769621505
Mean Per-Class Error: 0.34818673539768397
AUC: 0.684212436925981
AUCPR: 0.8280141965080744
Gini: 0.368424873851962

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3074193000474594: 

Maximum Metrics: Maximum metrics at their respective thresholds

Gains/Lift Table: Avg response rate: 67.47 %, avg score: 62.00 %

,,0,1,Error,Rate
0,0,0.0,163.0,1.0,(163.0/163.0)
1,1,0.0,338.0,0.0,(0.0/338.0)
2,Total,0.0,501.0,0.3253,(163.0/501.0)


,metric,threshold,value,idx
0,max f1,0.307419,0.805721,399.0
1,max f2,0.307419,0.912035,399.0
2,max f0point5,0.574889,0.771567,251.0
3,max accuracy,0.520049,0.700599,302.0
4,max precision,0.868887,1.000000,0.0
5,max recall,0.307419,1.000000,399.0
6,max specificity,0.868887,1.000000,0.0
7,max absolute_mcc,0.574889,0.298774,251.0
8,max min_per_class_accuracy,0.625125,0.642012,206.0
9,max mean_per_class_accuracy,0.574889,0.651813,251.0


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.011976,0.863534,1.482249,1.482249,1.00,0.865159,1.000000,0.865159,0.017751,0.017751,48.224852,48.224852
1,,2,0.021956,0.863278,1.482249,1.482249,1.00,0.863379,1.000000,0.864349,0.014793,0.032544,48.224852,48.224852
2,,3,0.031936,0.862149,1.482249,1.482249,1.00,0.862648,1.000000,0.863818,0.014793,0.047337,48.224852,48.224852
3,,4,0.041916,0.855305,1.482249,1.482249,1.00,0.858860,1.000000,0.862637,0.014793,0.062130,48.224852,48.224852
4,,5,0.051896,0.810482,1.482249,1.482249,1.00,0.831521,1.000000,0.856653,0.014793,0.076923,48.224852,48.224852
5,,6,0.101796,0.794512,1.363669,1.424121,0.92,0.799018,0.960784,0.828400,0.068047,0.144970,36.366864,42.412113
6,,7,0.151697,0.762226,1.482249,1.443242,1.00,0.783582,0.973684,0.813658,0.073964,0.218935,48.224852,44.324198
7,,8,0.201597,0.721776,1.185799,1.379518,0.80,0.736185,0.930693,0.794481,0.059172,0.278107,18.579882,37.951843
8,,9,0.301397,0.688004,0.859704,1.207394,0.58,0.705809,0.814570,0.765119,0.085799,0.363905,-14.029586,20.739449
9,,10,0.401198,0.667744,1.096864,1.179899,0.74,0.676464,0.796020,0.743066,0.109467,0.473373,9.686391,17.989932


In [13]:
from h2o.estimators import H2OXGBoostEstimator

top_xgb = H2OXGBoostEstimator(**extract_params_from_model(best_xgb_model.actual_params))

top_xgb.train(x=x, y=y, training_frame=train, validation_frame=test)

# h2o.save_model(top_xgb, MODELS_LOCATION + "FINAL/top_xgb")

test_perf = top_xgb.model_performance(test)

print('AUC on test data: ', test_perf.auc(), "\n\n============================")

test_perf

xgboost Model Build progress: |███████████████████████████████████████████| 100%
AUC on test data:  0.7161487639307366 


ModelMetricsBinomial: xgboost
** Reported on test data. **

MSE: 0.20783247115680908
RMSE: 0.45588646739819894
LogLoss: 0.5926501455155964
Mean Per-Class Error: 0.310197117653465
AUC: 0.7161487639307366
AUCPR: 0.8582261180336427
Gini: 0.43229752786147313

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.19317305088043213: 


,,0,1,Error,Rate
0,0,22.0,141.0,0.865,(141.0/163.0)
1,1,11.0,327.0,0.0325,(11.0/338.0)
2,Total,33.0,468.0,0.3034,(152.0/501.0)



Maximum Metrics: Maximum metrics at their respective thresholds

Gains/Lift Table: Avg response rate: 67.47 %, avg score: 62.81 %



,metric,threshold,value,idx
0,max f1,0.193173,0.811414,371.0
1,max f2,0.093578,0.912035,399.0
2,max f0point5,0.727320,0.776294,168.0
3,max accuracy,0.207489,0.698603,364.0
4,max precision,0.987254,1.000000,0.0
5,max recall,0.093578,1.000000,399.0
6,max specificity,0.987254,1.000000,0.0
7,max absolute_mcc,0.752970,0.363459,155.0
8,max min_per_class_accuracy,0.647942,0.625767,210.0
9,max mean_per_class_accuracy,0.752970,0.689803,155.0


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.011976,0.978517,1.482249,1.482249,1.00,0.983545,1.000000,0.983545,0.017751,0.017751,48.224852,48.224852
1,,2,0.021956,0.974664,1.482249,1.482249,1.00,0.975704,1.000000,0.979981,0.014793,0.032544,48.224852,48.224852
2,,3,0.031936,0.969281,1.482249,1.482249,1.00,0.970923,1.000000,0.977150,0.014793,0.047337,48.224852,48.224852
3,,4,0.041916,0.965367,1.482249,1.482249,1.00,0.967103,1.000000,0.974758,0.014793,0.062130,48.224852,48.224852
4,,5,0.051896,0.961167,1.482249,1.482249,1.00,0.963433,1.000000,0.972580,0.014793,0.076923,48.224852,48.224852
5,,6,0.101796,0.922065,1.422959,1.453185,0.96,0.935317,0.980392,0.954314,0.071006,0.147929,42.295858,45.318482
6,,7,0.151697,0.891542,1.422959,1.443242,0.96,0.908866,0.973684,0.939364,0.071006,0.218935,42.295858,44.324198
7,,8,0.201597,0.852796,1.422959,1.438221,0.96,0.871610,0.970297,0.922593,0.071006,0.289941,42.295858,43.822134
8,,9,0.301397,0.805159,1.215444,1.364454,0.82,0.829867,0.920530,0.891889,0.121302,0.411243,21.544379,36.445394
9,,10,0.401198,0.750237,1.126509,1.305264,0.76,0.780205,0.880597,0.864107,0.112426,0.523669,12.650888,30.526362


In [14]:
from h2o.estimators import H2ODeepLearningEstimator

top_dl = H2ODeepLearningEstimator(**extract_params_from_model(best_dl_model.actual_params))

top_dl.train(x=x, y=y, training_frame=train, validation_frame=test)

# h2o.save_model(top_dl, MODELS_LOCATION + "FINAL/top_dl")

test_perf = top_dl.model_performance(test)
print('AUC on test data: ', test_perf.auc(), "\n\n============================")

test_perf

deeplearning Model Build progress: |██████████████████████████████████████| 100%
AUC on test data:  0.665762151958471 


ModelMetricsBinomial: deeplearning
** Reported on test data. **

MSE: 0.31177726190482874
RMSE: 0.558370183574328
LogLoss: 2.4629902288506536
Mean Per-Class Error: 0.3506189421715613
AUC: 0.665762151958471
AUCPR: 0.8040539980448105
Gini: 0.331524303916942

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.0036193873092314976: 


,,0,1,Error,Rate
0,0,9.0,154.0,0.9448,(154.0/163.0)
1,1,6.0,332.0,0.0178,(6.0/338.0)
2,Total,15.0,486.0,0.3194,(160.0/501.0)



Maximum Metrics: Maximum metrics at their respective thresholds

Gains/Lift Table: Avg response rate: 67.47 %, avg score: 89.13 %



,metric,threshold,value,idx
0,max f1,3.619387e-03,0.805825,384.0
1,max f2,1.193808e-09,0.912035,399.0
2,max f0point5,9.996467e-01,0.753323,153.0
3,max accuracy,5.126684e-03,0.680639,382.0
4,max precision,9.999998e-01,0.892308,4.0
5,max recall,1.193808e-09,1.000000,399.0
6,max specificity,1.000000e+00,0.975460,0.0
7,max absolute_mcc,9.998716e-01,0.281183,126.0
8,max min_per_class_accuracy,9.994445e-01,0.624260,168.0
9,max mean_per_class_accuracy,9.998716e-01,0.649381,126.0


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.027944,1.000000e+00,1.270499,1.270499,0.857143,1.000000,0.857143,1.000000,0.035503,0.035503,27.049873,27.049873
1,,2,0.031936,1.000000e+00,0.741124,1.204327,0.500000,1.000000,0.812500,1.000000,0.002959,0.038462,-25.887574,20.432692
2,,3,0.041916,1.000000e+00,1.185799,1.199915,0.800000,1.000000,0.809524,1.000000,0.011834,0.050296,18.579882,19.991547
3,,4,0.051896,1.000000e+00,1.482249,1.254210,1.000000,1.000000,0.846154,1.000000,0.014793,0.065089,48.224852,25.421029
4,,5,0.101796,9.999999e-01,1.422959,1.336930,0.960000,1.000000,0.901961,1.000000,0.071006,0.136095,42.295858,33.693004
5,,6,0.151697,9.999996e-01,1.245089,1.306719,0.840000,1.000000,0.881579,1.000000,0.062130,0.198225,24.508876,30.671909
6,,7,0.201597,9.999987e-01,1.304379,1.306140,0.880000,0.999999,0.881188,1.000000,0.065089,0.263314,30.437870,30.613979
7,,8,0.301397,9.999929e-01,1.126509,1.246659,0.760000,0.999996,0.841060,0.999999,0.112426,0.375740,12.650888,24.665935
8,,9,0.401198,9.999442e-01,1.156154,1.224146,0.780000,0.999976,0.825871,0.999993,0.115385,0.491124,15.615385,22.414554
9,,10,0.500998,9.996709e-01,1.037574,1.186980,0.700000,0.999853,0.800797,0.999965,0.103550,0.594675,3.757396,18.697989


In [15]:
from h2o.estimators import H2ORandomForestEstimator

top_drf = H2ORandomForestEstimator(**extract_params_from_model(best_drf_model.actual_params, 
                                                                extra_params=['weights_column'],
                                                                additional_keys= {'nfolds':5,
                                                                                  'fold_assignment':'random'}))


top_drf.train(x=x, y=y, training_frame=train, validation_frame=test)

# h2o.save_model(top_drf, MODELS_LOCATION + "FINAL/top_drf")

test_perf = top_drf.model_performance(test)
print('AUC on test data: ', test_perf.auc(), "\n\n============================")

test_perf

drf Model Build progress: |███████████████████████████████████████████████| 100%
AUC on test data:  0.5949649689621375 


ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.2169740929087702
RMSE: 0.4658047798260235
LogLoss: 0.6232817135111628
Mean Per-Class Error: 0.417640759429339
AUC: 0.5949649689621375
AUCPR: 0.7692351352813652
Gini: 0.18992993792427493

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4043096005966431: 


,,0,1,Error,Rate
0,0,0.0,163.0,1.0,(163.0/163.0)
1,1,0.0,338.0,0.0,(0.0/338.0)
2,Total,0.0,501.0,0.3253,(163.0/501.0)



Maximum Metrics: Maximum metrics at their respective thresholds

Gains/Lift Table: Avg response rate: 67.47 %, avg score: 65.47 %



,metric,threshold,value,idx
0,max f1,0.404310,0.805721,399.0
1,max f2,0.404310,0.912035,399.0
2,max f0point5,0.553318,0.721703,320.0
3,max accuracy,0.404310,0.674651,399.0
4,max precision,0.824123,1.000000,0.0
5,max recall,0.404310,1.000000,399.0
6,max specificity,0.824123,1.000000,0.0
7,max absolute_mcc,0.771020,0.217416,36.0
8,max min_per_class_accuracy,0.648860,0.553254,194.0
9,max mean_per_class_accuracy,0.748378,0.582359,71.0


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.029940,0.802538,1.284615,1.284615,0.866667,0.806609,0.866667,0.806609,0.038462,0.038462,28.461538,28.461538
1,,2,0.031936,0.801635,1.482249,1.296967,1.000000,0.801635,0.875000,0.806298,0.002959,0.041420,48.224852,29.696746
2,,3,0.043912,0.801105,1.482249,1.347499,1.000000,0.801193,0.909091,0.804906,0.017751,0.059172,48.224852,34.749866
3,,4,0.051896,0.796630,1.482249,1.368229,1.000000,0.797912,0.923077,0.803830,0.011834,0.071006,48.224852,36.822940
4,,5,0.109780,0.781151,1.431137,1.401399,0.965517,0.786982,0.945455,0.794946,0.082840,0.153846,43.113650,40.139860
5,,6,0.151697,0.765917,1.129332,1.326222,0.761905,0.772177,0.894737,0.788655,0.047337,0.201183,12.933221,32.622236
6,,7,0.203593,0.752417,0.969162,1.235207,0.653846,0.758878,0.833333,0.781065,0.050296,0.251479,-3.083751,23.520710
7,,8,0.301397,0.735545,0.907499,1.128865,0.612245,0.742921,0.761589,0.768687,0.088757,0.340237,-9.250091,12.886477
8,,9,0.405190,0.713185,0.997667,1.095258,0.673077,0.724496,0.738916,0.757367,0.103550,0.443787,-0.233273,9.525753
9,,10,0.502994,0.658277,0.998249,1.076395,0.673469,0.691041,0.726190,0.744470,0.097633,0.541420,-0.175100,7.639476


### Check for the best  meta-learner

In [18]:
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator

collection_of_models = [
                        # top_nb,
                        # top_glm,

                        # checkpoint-enabled models
                        # top_gbm, # based on boosting, like XGB
                        top_xgb,
                        top_dl,
                        top_drf]


meta_algos = ["auto", "xgboost", "drf", "gbm", "glm", "naivebayes", "deeplearning"]

ensemble_list = []

for metalearner in meta_algos:
    print("\n\n>>>>> ", metalearner, " <<<<<<")

    ensemble = H2OStackedEnsembleEstimator(
                                       base_models= collection_of_models,

                                    #    model_id= "stacked_ensemble_ALL_FEATURES_ALL_MODELS_metalearner_" + metalearner,
                                    #    model_id= "stacked_ensemble_ALL_FEATURES_CHECKPOINT_MODELS_metalearner_" + metalearner,
                                       model_id= "stacked_ensemble_ALL_FEATURES_CHECKPOINT_nogbm_MODELS_metalearner_" + metalearner,

                                       metalearner_algorithm= metalearner,

                                       #metalearner_params
                                       )
                                       
    ensemble.train(x=x, y=y, training_frame=train, validation_frame=test)

    # h2o.save_model(ensemble, MODELS_LOCATION + "FINAL/top_ensemble_ALL_MODELS_METALEARNER_" + metalearner)

    # h2o.save_model(ensemble, MODELS_LOCATION + "FINAL/top_ensemble_CHECKPOINT_MODELS_METALEARNER_" + metalearner)

    h2o.save_model(ensemble, MODELS_LOCATION + "FINAL/top_ensemble_CHECKPOINT_nogbm_MODELS_METALEARNER_" + metalearner)


    print("AUC on test data: ",  ensemble.model_performance(valid=True).auc())

    ensemble_list.append(ensemble)








>>>>>  auto  <<<<<<
stackedensemble Model Build progress: |███████████████████████████████████| 100%
AUC on test data:  0.7199150542708824


>>>>>  xgboost  <<<<<<
stackedensemble Model Build progress: |███████████████████████████████████| 100%
AUC on test data:  0.7031800196028606


>>>>>  drf  <<<<<<
stackedensemble Model Build progress: |███████████████████████████████████| 100%
AUC on test data:  0.7096507786691836


>>>>>  gbm  <<<<<<
stackedensemble Model Build progress: |███████████████████████████████████| 100%
AUC on test data:  0.7027171742839511


>>>>>  glm  <<<<<<
stackedensemble Model Build progress: |███████████████████████████████████| 100%
AUC on test data:  0.7168384942098959


>>>>>  naivebayes  <<<<<<
stackedensemble Model Build progress: |███████████████████████████████████| 100%
AUC on test data:  0.7033161505790104


>>>>>  deeplearning  <<<<<<
stackedensemble Model Build progress: |███████████████████████████████████| 100%
AUC on test data:  0.7169201727955857